# LoRA Model Training in Google Colab

This notebook is designed to train a LoRA model using the provided dataset and the training script from `train_lora.py`. Follow the steps below to set up the environment and execute the training process.

In [4]:
!pip install -q peft transformers datasets accelerate

In [5]:
# Load the dataset
import json

with open('merged_min.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

print(f"Loaded {data} examples.")
from datasets import Dataset

dataset = Dataset.from_list(data)
print(f'Dataset loaded with {len(dataset)} examples.')

Loaded [{'instruction': 'USERS', 'input': 'CHAPTER 1: SCALE FROM ZERO TO MILLIONS OF\nUSERS\nDesigning a system that supports millions of users is challenging, and it is a journey that\nrequires continuous refinement and endless improvement. In this chapter, we build a system\nthat supports a single user and gradually scale it up to serve millions of users. After reading\nthis chapter, you will master a handful of techniques that will help you to crack the system\ndesign interview questions.', 'output': 'Single server setup\nA journey of a thousand miles begins with a single step, and building a complex system is no\ndifferent. To start with something simple, everything is running on a single server. Figure 1-1\nshows the illustration of a single server setup where everything is running on one server: web\napp, database, cache, etc.\nTo understand this setup, it is helpful to investigate the request flow and traffic source. Let us\nfirst look at the request flow (Figure 1-2).', 'metada

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

import sys
sys.modules["bitsandbytes"] = None
from peft import get_peft_model, LoraConfig, TaskType
class NoToTrainer(Trainer):
    def _move_model_to_device(self, model, device):
        # Пропускаємо переміщення моделі
        print(f"[NoToTrainer] Пропускаємо model.to({device})")
        return model
# 💾 Використовуємо GPU + CPU offload
model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # device_map="auto",  # це важливо для offloading
    torch_dtype=torch.bfloat16,
    offload_folder="./offload",
    low_cpu_mem_usage=True
)

# 🪛 Вмикаємо потрібні режими
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.config.use_cache = False

# 🔧 LoRA конфіг
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)
model = model.to("cuda")
def format_example(example):
    source = example.get("metadata", {}).get("source", "")
    output = example["output"]
    if source and output:
        output += f"\n\nДетальніше: {source}"

    text = f"### Інструкція:\n{example['instruction']}\n\n### Вхід:\n{example['input']}\n\n### Вихід:\n{output}"

    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )

    return {
        "input_ids": tokenized["input_ids"][0],
        "attention_mask": tokenized["attention_mask"][0],
        "labels": tokenized["input_ids"][0]
    }

tokenized_dataset = dataset.map(format_example)

# ⚙️ Training arguments
training_args = TrainingArguments(
    output_dir="./lora_mistral",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    logging_dir="./logs",
    bf16=True,           # або fp16=True, якщо GPU не підтримує bf16
    optim="adamw_torch"
)

trainer = NoToTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

# 🚀 Go
trainer.train()

# 💾 Save result
model.save_pretrained("lora_mistral_adapter")
tokenizer.save_pretrained("lora_mistral_adapter")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

[NoToTrainer] Пропускаємо model.to(cuda:0)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: bmo-ossystem (bmo-ossystem-os-system) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
